[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EMQA_garch_electricity/EMQA_garch_electricity.ipynb)

# EMQA_garch_electricity

**Rolling 1-step-ahead GARCH(1,1) forecast with confidence intervals on electricity prices.**

- Load hourly electricity data, resample to daily mean
- Compute log returns
- 80/20 train/test split
- Rolling 1-step-ahead GARCH(1,1) with expanding window
- Point forecast: random walk (last observed price)
- 95% CI: last_price +/- 1.96 * sigma_{t+1} * last_price

**Output:** `garch_electricity.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")

In [ ]:
# Load hourly electricity data and resample to daily mean
url = 'https://raw.githubusercontent.com/QuantLet/EMQA/main/EMQA_garch_electricity/electricity_cache.csv'
elec = pd.read_csv(url, parse_dates=[0], index_col=0)

if 'price' in elec.columns:
    price_col = 'price'
elif 'Price' in elec.columns:
    price_col = 'Price'
else:
    price_col = elec.columns[0]

# Resample hourly to daily mean price
daily_price = elec[price_col].resample('D').mean().dropna()
print(f'Loaded {len(elec)} hourly observations -> {len(daily_price)} daily observations')
print(f'Date range: {daily_price.index[0].date()} to {daily_price.index[-1].date()}')
print(f'Mean: {daily_price.mean():.2f}, Std: {daily_price.std():.2f}')

In [ ]:
# Compute log returns (percent)
log_returns = (np.log(daily_price / daily_price.shift(1)) * 100).dropna()

# Winsorize extreme outliers for numerical stability
clip_val = log_returns.abs().quantile(0.995)
log_returns = log_returns.clip(-clip_val, clip_val)

# 80/20 train/test split
split = int(len(log_returns) * 0.80)
train_ret = log_returns.iloc[:split]
test_ret = log_returns.iloc[split:]

# Align prices with returns index (shifted by 1 due to differencing)
prices_aligned = daily_price.loc[log_returns.index]
train_prices = prices_aligned.iloc[:split]
test_prices = prices_aligned.iloc[split:]

print(f"Log returns: {len(log_returns)} obs, std={log_returns.std():.2f}%")
print(f"Train: {len(train_ret)} obs ({train_ret.index[0].date()} to {train_ret.index[-1].date()})")
print(f"Test:  {len(test_ret)} obs ({test_ret.index[0].date()} to {test_ret.index[-1].date()})")

In [ ]:
# Rolling 1-step-ahead GARCH(1,1) forecast with expanding window
from arch import arch_model

forecasted_vol = []      # sigma_{t+1} in % return space
price_forecast = []      # random walk: last observed price
ci_lower_price = []      # lower 95% CI on price
ci_upper_price = []      # upper 95% CI on price

for i in range(len(test_ret)):
    # Expanding window: all returns up to the current test point
    history = log_returns.iloc[:split + i]

    try:
        am = arch_model(history, vol='Garch', p=1, q=1, dist='normal', mean='Constant')
        res = am.fit(disp='off', show_warning=False)
        fc = res.forecast(horizon=1)
        sigma_next = np.sqrt(fc.variance.values[-1, 0])  # sigma_{t+1} in %
    except Exception:
        # Fallback: use sample std of recent window
        sigma_next = history.iloc[-60:].std()

    forecasted_vol.append(sigma_next)

    # Point forecast for price: random walk (last observed price)
    last_price = prices_aligned.iloc[split + i - 1]
    price_forecast.append(last_price)

    # 95% CI: last_price +/- 1.96 * sigma_{t+1}/100 * last_price
    margin = 1.96 * (sigma_next / 100) * last_price
    ci_lower_price.append(last_price - margin)
    ci_upper_price.append(last_price + margin)

    if (i + 1) % 50 == 0:
        print(f"  Rolling GARCH forecast: {i+1}/{len(test_ret)} done")

forecasted_vol = pd.Series(forecasted_vol, index=test_ret.index)
price_forecast = pd.Series(price_forecast, index=test_ret.index)
ci_lower_price = pd.Series(ci_lower_price, index=test_ret.index)
ci_upper_price = pd.Series(ci_upper_price, index=test_ret.index)

print(f"\nRolling GARCH forecast complete: {len(forecasted_vol)} predictions")

In [ ]:
# Evaluation metrics (price-level forecast)
actual_prices = test_prices.values
pred_prices = price_forecast.values

mae = np.mean(np.abs(actual_prices - pred_prices))
rmse = np.sqrt(np.mean((actual_prices - pred_prices)**2))
mape = np.mean(np.abs((actual_prices - pred_prices) / actual_prices)) * 100
ss_res = np.sum((actual_prices - pred_prices)**2)
ss_tot = np.sum((actual_prices - actual_prices.mean())**2)
r2 = 1 - ss_res / ss_tot

dir_actual = np.sign(np.diff(actual_prices))
dir_pred = np.sign(np.diff(pred_prices))
direction_acc = np.mean(dir_actual == dir_pred) * 100

# CI coverage
coverage = np.mean((actual_prices >= ci_lower_price.values) &
                    (actual_prices <= ci_upper_price.values)) * 100

print("Out-of-Sample Forecast Metrics (Random Walk + GARCH CI)")
print("=" * 55)
print(f"R2             = {r2:.4f}")
print(f"MAE            = {mae:.2f}")
print(f"RMSE           = {rmse:.2f}")
print(f"MAPE           = {mape:.2f}%")
print(f"Direction Acc.  = {direction_acc:.1f}%")
print(f"95% CI Coverage = {coverage:.1f}%")

# --- Plot 1: Returns with conditional volatility bands ---
fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=False)

# Panel 1 - Returns with +/-1.96*sigma bands (test period)
ax1 = axes[0]
ax1.plot(test_ret.index, test_ret.values, color=COLORS['gray'], linewidth=0.5,
         alpha=0.7, label='Log Returns (%)')
ax1.plot(forecasted_vol.index, 1.96 * forecasted_vol.values, color=COLORS['red'],
         linewidth=1.2, label='+1.96$\\sigma$')
ax1.plot(forecasted_vol.index, -1.96 * forecasted_vol.values, color=COLORS['red'],
         linewidth=1.2, label='$-$1.96$\\sigma$')
ax1.fill_between(forecasted_vol.index, -1.96 * forecasted_vol.values,
                 1.96 * forecasted_vol.values, color=COLORS['red'], alpha=0.08)
ax1.set_ylabel('Returns (%)')
ax1.set_title('Electricity Log Returns with GARCH(1,1) Conditional Volatility Bands',
              fontsize=13, fontweight='bold')
ax1.legend(loc='upper center', bbox_to_anchor=(0.5, -0.08), frameon=False, ncol=3)

# Panel 2 - Price forecast with CI
ax2 = axes[1]
ax2.plot(test_prices.index, test_prices.values, color=COLORS['blue'], linewidth=1.2,
         label='Actual Price')
ax2.plot(price_forecast.index, price_forecast.values, color=COLORS['red'], linewidth=1.2,
         label='Forecast (Random Walk)')
ax2.fill_between(test_prices.index, ci_lower_price.values, ci_upper_price.values,
                 color=COLORS['red'], alpha=0.15, label='95% CI (GARCH)')
ax2.set_ylabel('Price')
ax2.set_xlabel('Date')
ax2.set_title(f'Electricity Price: Rolling GARCH Forecast (R$^2$={r2:.3f}, Coverage={coverage:.0f}%)',
              fontsize=13, fontweight='bold')
ax2.legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False, ncol=3)

fig.tight_layout()
save_fig(fig, 'garch_electricity.pdf')
plt.show()